In [1]:
import math
import torch
from diffusion_edf.dist import *
from diffusion_edf import transforms
import plotly.graph_objs as go

In [2]:
dtype = torch.float64
device = 'cpu'

eps = 1e-8
q = sample_igso3(eps = eps, N=3, dtype=dtype, device=device)
# theta = 0.4
# # q = torch.tensor([[math.cos(theta/2), math.sin(theta/2), 0., 0.]], dtype=dtype, device=device)
# randomaxis = torch.randn(1,3, dtype=dtype, device=device)
# randomaxis = randomaxis / randomaxis.norm(dim=-1, keepdim=True) * math.sin(theta/2)
# q = torch.cat([torch.ones_like(randomaxis[...,:1]) * math.cos(theta/2), randomaxis], dim=-1)

log_prob = torch.log(igso3(q, eps))
score = igso3_score(q, eps)
print(f"Anaylitic score: {score}")

dt = 0.0000001

log_prob_rot = []
for i in range(3):
    qrot = transforms.quaternion_multiply(q,transforms.axis_angle_to_quaternion(dt * torch.eye(3, device=q.device, dtype=q.dtype)[i]))
    log_prob_rot.append(torch.log(igso3(qrot, eps)))

log_prob_rot = torch.stack(log_prob_rot, dim=-1)
print(f"Numerical score: {(log_prob_rot - log_prob[...,None]) / dt}")
print(f"Allclose: {torch.allclose((log_prob_rot - log_prob[...,None]) / dt, score, atol=0, rtol=1e-2)}")

Anaylitic score: tensor([[ -6531.3837,   4498.5191,   2586.6971],
        [-12923.0590,   6868.0829,  -5937.9300],
        [  9641.9716, -14920.2634, -13073.0142]], dtype=torch.float64)
Numerical score: tensor([[ -6533.8330,   4496.0201,   2584.3512],
        [-12925.4222,   6865.7059,  -5940.2907],
        [  9639.4419, -14922.7480, -13075.5370]], dtype=torch.float64)
Allclose: True
